# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, I will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on lj_speech dataset. Note that in this notebook, I will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, I fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [li_speech](https://huggingface.co/datasets/lj_speech) dataset.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition.

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, one has a much easier time getting access to a V100 or P100 GPU however.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 10 10:58:40 2022       

+-----------------------------------------------------------------------------+

| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |

|-------------------------------+----------------------+----------------------+

| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |

| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |

|                               |                      |               MIG M. |

|===============================+======================+======================|

|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |

| N/A   38C    P0    41W / 300W |   7183MiB / 16160MiB |      0%      Default |

|                               |                      |                  N/A |

+-------------------------------+----------------------+----------------------+

                                                                            

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [1]:
%%capture
# !pip install datasets==1.18.3
!pip install datasets==2.15
!pip install transformers==4.17.0
!pip install jiwer


Then you need to install Git-LFS to upload your model checkpoints:

In [2]:
%%capture
!apt install git-lfs




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

In [3]:
pip install numpy==1.23.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
albumentations 1.4.0 requires numpy>=1.24.4, but you have numpy 1.23.4 which is incompatible.
apache-beam 2.46.0 requires d

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

In [4]:
from datasets import load_dataset, load_metric

timit = load_dataset("lj_speech")

Generating train split:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [5]:
timit

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 13100
    })
})

In [6]:
from datasets import DatasetDict
import numpy as np

# Assuming timit is your DatasetDict
timit_train_size = int(len(timit['train']) * 0.8)  # 80% for training
timit_test_size = len(timit['train']) - timit_train_size  # Remaining 20% for testing

# Get the indices for splitting the data
indices = np.arange(len(timit['train']))
np.random.shuffle(indices)

train_indices = indices[:timit_train_size]
test_indices = indices[timit_train_size:]

# Create train and test datasets
train_dataset = timit['train'].select(train_indices)
test_dataset = timit['train'].select(test_indices)

# Create a new DatasetDict for train and test datasets
timit = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})



In [7]:
timit

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 10480
    })
    test: Dataset({
        features: ['id', 'audio', 'file', 'text', 'normalized_text'],
        num_rows: 2620
    })
})

Many ASR datasets only provide the target text, `'text'` for each audio `'audio'` and file `'file'`. lj_speech actually provides much more information about each audio file, such as the `'normalized_text'`, etc. However, we want to keep the notebook as general as possible, so that we will only consider the transcribed text for fine-tuning.



In [ ]:
timit = timit.remove_columns(["id", "normalized_text"])

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)

,text
0,"Valuable space was cumbered with many long and winding passages, numerous staircases, and unnecessarily thick and cumbrous inner walls."
1,"He carried this character with him into penal servitude, and after enduring the full severity of his punishment for several years,"
2,learnt through the firm's correspondence that a quantity of gold-dust
3,"""Money Moses"" and Mrs. Abrahams imposed upon Moss as to the price paid by Solomons;"
4,"She did not speak to him when she joined him there, although she thought that he was still awake."
5,Arrange the rolls in regular rows in the pan about a quarter of an inch apart.
6,There remains the question of what impelled Oswald to conceive and to carry out the assassination of the President of the United States.
7,".45 caliber pistol, and 12-gauge riot gun."
8,"According to Fauntleroy's own case, he found at once that the firm was heavily involved,"
9,"Even Mr. Cope admitted that the committal of this class of prisoners to Newgate was most inconvenient,"


Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [lj_speech](https://huggingface.co/datasets/lj_speech) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [11]:
import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\ü\è\â\à\é\ê]'

# def remove_special_characters(batch):
#     batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
#     return batch

# Define the regex pattern to include characters to be ignored
chars_to_ignore_regex = '[^a-zA-Z\s]'

def remove_special_characters(batch):
    # Use re.sub with the negated regex pattern to remove non-alphabetic characters except spaces
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch


In [12]:
timit = timit.map(remove_special_characters)

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [13]:
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

,text
0,although the strain lasted fourteen days he showed no signs of exhaustion either physical or mental
1,but we cannot yield our constitutional destiny to the personal judgment of a few men who being fearful of the future
2,it was his own handiwork a bird pecking at a flower
3,there was no decency whatever in the internal arrangements
4,oswald never asked mrs paine about the use of curtain rods
5,even where cells had been built
6,wait after that seven days and post me another detachment two thousand strong at the nineveh gates
7,so much concerns the general plan of skeletal structures and is strongly suggestive of in fact it is inexplicable without common origin
8,he was then shown two photographs of oswald and he advised that oswald was quote unquestionably end quote the man he saw
9,entertained literary artistic and dramatic celebrities had a smart turn out attended all the racemeetings and dressed in the latest fashion


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [16]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'s': 0,
 'z': 1,
 'v': 2,
 'd': 3,
 'f': 4,
 'i': 5,
 'j': 6,
 'x': 7,
 'e': 8,
 'r': 9,
 ' ': 10,
 'k': 11,
 'u': 12,
 'w': 13,
 'y': 14,
 't': 15,
 'g': 16,
 'c': 17,
 'o': 18,
 'p': 19,
 'q': 20,
 'a': 21,
 'h': 22,
 'l': 23,
 'n': 24,
 'b': 25,
 'm': 26}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because:

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set.

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

Cool, now our vocabulary is complete and consists of 29 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 29.

Let's now save the vocabulary as a json file.

In [20]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-base-timit-demo-colab`

### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus,
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, [Timit](hhtps://huggingface.co/datasets/timit_asr), was luckily also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining.



A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `'text'`, our datasets include two more column names `'file'` and `'audio'`. `'file'` states the absolute path of the audio file. Let's take a look.

In [24]:
timit["train"][999]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ014-0310.wav'

`Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

In [25]:
timit["train"][999]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ014-0310.wav',
 'array': array([-0.00067139, -0.00054932, -0.00015259, ...,  0.00057983,
         0.00067139,  0.00076294]),
 'sampling_rate': 22050}

The sampling rate is 22050 which is not what `Wav2Vec2` expects as an input. All the audio are resampled later in the code.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [74]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

the aged but wary kerp the moment the plot was discovered vanished and was never more heard of 


It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [27]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: on the th floor of the hospital mrs kennedy and the attorney general joined other members of the kennedy family to await the conclusion of the autopsy 
Input array shape: (196509,)
Sampling rate: 22050


Finally, we can process the dataset to the format expected by the model for training. We will make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [28]:
import librosa

# Define a function to resample the audio to 16000 Hz
def resample_audio(audio_array, current_sr, target_sr=16000):
    return librosa.resample(audio_array, orig_sr=current_sr, target_sr=target_sr)

def prepare_dataset(batch):
    audio = batch["audio"]

    # Resample the audio if the current sampling rate is not 16000
    if audio["sampling_rate"] != 16000:
        audio_array = resample_audio(audio["array"], audio["sampling_rate"])
    else:
        audio_array = audio["array"]

    # Process the resampled audio
    batch["input_values"] = processor(audio_array, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [29]:
timit = timit.map(prepare_dataset)

Map:   0%|          | 0/10480 [00:00<?, ? examples/s]

2024-03-28 07:25:14.998078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 07:25:14.998214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 07:25:15.141620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training.

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [30]:
from huggingface_hub import notebook_login

notebook_login()

Awesome, now we are ready to start training!

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [31]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [33]:
wer_metric = load_metric("wer")

/tmp/ipykernel_34/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [34]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [35]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [36]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-model-training",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=5,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/keyi10/wav2vec2-model-training into local empty directory.




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 270 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 16 or even less and eventually make use of [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [39]:
!pip install numpy==1.23.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 61.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
albumentations 1.4.0 requires numpy>=1.24.4, but you have numpy 1.23.1 which is incompatible.
apache-beam 2.46.0 requires d

In [40]:
import numpy as np
np.bool = np.bool_

In [41]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: audio, input_length, text, file. If audio, input_length, text, file are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10480
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1640
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B serv

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.509900,1.132345,0.674018
1000,0.329300,0.142994,0.185056
1500,0.104700,0.125754,0.144556


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: audio, input_length, text, file. If audio, input_length, text, file are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to wav2vec2-model-training/checkpoint-500
Configuration saved in wav2vec2-model-training/checkpoint-500/config.json
Model weights saved in wav2vec2-model-training/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-model-training/checkpoint-500/preprocessor_config.json
Feature extractor saved in wav2vec2-model-training/preprocessor_config.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To ma

TrainOutput(global_step=1640, training_loss=1.2084761456745428, metrics={'train_runtime': 8128.5489, 'train_samples_per_second': 6.446, 'train_steps_per_second': 0.202, 'total_flos': 3.164539011632732e+18, 'train_loss': 1.2084761456745428, 'epoch': 5.0})

In [47]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length, audio, file, text. If input_length, audio, file, text are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10480
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1640
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B serv

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.367700,0.961637,0.646809
1000,0.316700,0.155175,0.182910
1500,0.107200,0.092958,0.135301


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length, audio, file, text. If input_length, audio, file, text are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to kywav2vec2test/checkpoint-500
Configuration saved in kywav2vec2test/checkpoint-500/config.json
Model weights saved in kywav2vec2test/checkpoint-500/pytorch_model.bin
Feature extractor saved in kywav2vec2test/checkpoint-500/preprocessor_config.json
Feature extractor saved in kywav2vec2test/preprocessor_config.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=T

TrainOutput(global_step=1640, training_loss=1.1621675485517922, metrics={'train_runtime': 7323.7981, 'train_samples_per_second': 7.155, 'train_steps_per_second': 0.224, 'total_flos': 3.1661903612203776e+18, 'train_loss': 1.1621675485517922, 'epoch': 5.0})

In [44]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: text, input_length, audio, file. If text, input_length, audio, file are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10480
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6550
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.577400,1.763824,0.859915
1000,0.571200,0.279076,0.296488
1500,0.232600,0.200370,0.213510
2000,0.164600,0.138663,0.179117
2500,0.129300,0.161286,0.164332
3000,0.096900,0.113030,0.146530
3500,0.083900,0.081343,0.129025
4000,0.081200,0.077941,0.121758
4500,0.058600,0.071139,0.119770
5000,0.050800,0.077260,0.115679


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: text, input_length, audio, file. If text, input_length, audio, file are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /kaggle/working/checkpoint-500
Configuration saved in /kaggle/working/checkpoint-500/config.json
Model weights saved in /kaggle/working/checkpoint-500/pytorch_model.bin
Feature extractor saved in /kaggle/working/checkpoint-500/preprocessor_config.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer 

TrainOutput(global_step=6550, training_loss=0.39515475658970023, metrics={'train_runtime': 10252.1223, 'train_samples_per_second': 5.111, 'train_steps_per_second': 0.639, 'total_flos': 3.1569129504290534e+18, 'train_loss': 0.39515475658970023, 'epoch': 5.0})

In [46]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: text, audio, file, input_length. If text, audio, file, input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10480
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3930
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.499800,1.674284,0.891083
1000,0.546600,0.306582,0.314239
1500,0.242700,0.207288,0.223257
2000,0.164000,0.171065,0.190314
2500,0.136300,0.131266,0.157325
3000,0.088900,0.119272,0.143622
3500,0.075800,0.105108,0.133327


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: text, audio, file, input_length. If text, audio, file, input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /kaggle/working/checkpoint-500
Configuration saved in /kaggle/working/checkpoint-500/config.json
Model weights saved in /kaggle/working/checkpoint-500/pytorch_model.bin
Feature extractor saved in /kaggle/working/checkpoint-500/preprocessor_config.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer 

TrainOutput(global_step=3930, training_loss=0.612585601127178, metrics={'train_runtime': 6129.5382, 'train_samples_per_second': 5.129, 'train_steps_per_second': 0.641, 'total_flos': 1.8981156218495616e+18, 'train_loss': 0.612585601127178, 'epoch': 3.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [43]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-model-training
Configuration saved in wav2vec2-model-training/config.json
Model weights saved in wav2vec2-model-training/pytorch_model.bin
Feature extractor saved in wav2vec2-model-training/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 1.00/360M [00:00<?, ?B/s]

Upload file runs/Mar28_08-23-39_6073131fb751/events.out.tfevents.1711615221.6073131fb751.34.0:   0%|          …

To https://huggingface.co/keyi10/wav2vec2-model-training
   a5a378c..8009c1f  main -> main

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'lj_speech', 'type': 'lj_speech', 'args': 'main'}}
To https://huggingface.co/keyi10/wav2vec2-model-training
   8009c1f..29d2bb4  main -> main



'https://huggingface.co/keyi10/wav2vec2-model-training/commit/8009c1f9165b60a42a9289523e6c46569e197e39'

If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). Let's call the repo to which we will upload the files
`"wav2vec2-model-training"`:

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [46]:
tokenizer.push_to_hub("wav2vec2-model-training")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
tokenizer config file saved in wav2vec2-model-training/tokenizer_config.json
Special tokens file saved in wav2vec2-model-training/special_tokens_map.json
To https://huggingface.co/keyi10/wav2vec2-model-training
   29d2bb4..4dd1cda  main -> main



'https://huggingface.co/keyi10/wav2vec2-model-training/commit/4dd1cda4d4975a75dbe8dfb65e518e9d6e4d17f0'

Great, you can see the just created repository under `https://huggingface.co/keyi10/wav2vec2-model-training`

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

In [47]:
processor = Wav2Vec2Processor.from_pretrained("keyi10/wav2vec2-model-training")

loading feature extractor configuration file https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/d0b51c2f8cbf887110ddfef21574c0732a54c023a14912983e1a3ab6649e1240.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc2rfhb_6


Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

storing https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/75172bf7fa6d90ea962556a8479e1c3cf62a9634336d9d97c8bb7aa72b47f932.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
creating metadata file for /root/.cache/huggingface/transformers/75172bf7fa6d90ea962556a8479e1c3cf62a9634336d9d97c8bb7aa72b47f932.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpqjr9u69g


Downloading:   0%|          | 0.00/259 [00:00<?, ?B/s]

storing https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/f0d9899072f630f34043829e8e47b47ac819c8257547a666ec13c8c199048fd0.477c41b4ac51c4f25ca58115f0c1f6c46b0c52c8b788e6659a9151e9445275f4
creating metadata file for /root/.cache/huggingface/transformers/f0d9899072f630f34043829e8e47b47ac819c8257547a666ec13c8c199048fd0.477c41b4ac51c4f25ca58115f0c1f6c46b0c52c8b788e6659a9151e9445275f4
https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn9_el9ga


Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

storing https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/cbd650a48996dd4e4b5c6b90c549347fc728c647ada75b2117fc12269b044094.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
creating metadata file for /root/.cache/huggingface/transformers/cbd650a48996dd4e4b5c6b90c549347fc728c647ada75b2117fc12269b044094.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
loading file https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f0d9899072f630f34043829e8e47b47ac819c8257547a666ec13c8c199048fd0.477c41b4ac51c4f25ca58115f0c1f6c46b0c52c8b788e6659a9151e9445275f4
loading file https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/75172bf7fa6d90ea962556a8479e1c3cf62a9634336d9d97c8bb7aa72b47f932.59710b1a6a5501d31e746b6e464f5c44de

In [48]:
model = Wav2Vec2ForCTC.from_pretrained("keyi10/wav2vec2-model-training").cuda()

loading configuration file https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/02bb7db16490489bd4b1d1c58e9c8d2e4b442965eb23d37242017403bd8dd2d1.32d73d4594f2605c378c44b0c3fd8d2492ab4f361943b5ad6a0be88998434e52
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

storing https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/1909d063968d056ea50d92ec69f36c76a4157edf7a579689044ddb29063f67c6.91c69a7699e8f3f5f5890c7aa88074c95eb945a1a14a1ef5569f5fbc3c8b09ac
creating metadata file for /root/.cache/huggingface/transformers/1909d063968d056ea50d92ec69f36c76a4157edf7a579689044ddb29063f67c6.91c69a7699e8f3f5f5890c7aa88074c95eb945a1a14a1ef5569f5fbc3c8b09ac
loading weights file https://huggingface.co/keyi10/wav2vec2-model-training/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1909d063968d056ea50d92ec69f36c76a4157edf7a579689044ddb29063f67c6.91c69a7699e8f3f5f5890c7aa88074c95eb945a1a14a1ef5569f5fbc3c8b09ac
/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated i

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`.

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [49]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [50]:
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

In [58]:
results_2 = timit["test"].map(map_to_result)

Map:   0%|          | 0/2620 [00:00<?, ? examples/s]

Let's compute the overall WER now.

In [59]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results_2["pred_str"], references=results_2["text"])))

Test WER: 0.064


21.8% WER - not bad! Our demo model would have probably made it on the official [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit).

Let's take a look at some predictions to see what errors are made by the model.

In [67]:
show_random_elements(results_2.remove_columns(["audio", "file","input_values","input_length","labels"]))

,text,pred_str
0,in one or another government agency to clarify the division of authority between the agencies involved and to retain the existing system,in one or another government agency to clarify the division of authority between the agencies involved and to retain the existing system
1,it had been purchased in march and taken to new orleans where marina oswald saw it in their rented apartment,it had been purchased in march and taken to new orleans where marina oswald saw it in their rented apartment
2,the dryad was a brig owned principally by two persons named wallace one a seaman the other merchant,the driad was a brigg owned principaly by two persons named wallase one a seaman the other merchant
3,his effrontery was only outdone by his cool contempt for the consolations of religion,his effrumptery was only outdone by his col contempt for the consolations of religion
4,on the other hand he also told her that he wished his mother had been more firm with him in her attempts to get him to return to school,on the other hand he also told her that he wished his mother had been more firm with him in her attempts to get him to return to school
5,in the protection of the president if the presidential car permitted immediate access to the president by a secret service agent at the first sign of danger,in the protection of the president if the presidential car permitted immediate access to the president by a secret service agent at the first signe of danger
6,in the face of impediments confessedly discouraging,in the face of impediments confessidly discouraging
7,and his death was not compassed until the executioner terminated his sufferings by hanging on to his legs,and his death was not compased until the executioner terminated his sufferings by hanging on to his legs
8,his name headed every subscription list his purse was always open,his name headed every subscriptionlist his perse was always open
9,it seeks to maintain the federal bench in full vigor,it seeks to maintain the federal bench in full vigor


In [69]:
show_random_elements(results_2.remove_columns(["audio", "file","input_values","input_length","labels"]))

,text,pred_str
0,idleness was still the general rule for all prisoners in newgate in defiance of the law,idleness was still the general rule for all prisoners in newgate in defiance of the law
1,it sought to obtain and diffuse useful information,it sought to obtain and diffuse useful information
2,since the assassination the secret service and the department of the treasury,since the assassination the secret service and the department of the treasury
3,he staggers towards the pew reels into it stumbles forward flings himself on the ground and by a curious twist of the spine,he staggers towards the pew reals into it stumbles forward flings himself on the ground and by a curious twist of the spine
4,the second is to establish the practical means to help those who are unemployed in this present emergency,the second is to establish the practical means to help those who are unemployed in this present emergency
5,oswald experienced some difficulty finding employment perhaps this was partially because of his lack of any specific skill or training,oswald experienced some difficulty finding employment perhaps this was partially because of his lack of any specific skill or training
6,helen markham was screaming as she leaned over the body,hellon markham was screaming as she leaned over the body
7,nor did the methods by which they were perpetrated greatly vary from those in times past,nor did the methods by which they were perpetrated greatly vary from those in times passed
8,has hitherto been handled on an informal basis,has hitherto been handled on an informal basis
9,although his execution was imminent a surgeon attended to his wounds and he was carried more dead than alive to the scaffold,although his execution was imminent a surgeon attended to his wounds and he was carried more dead than alive to the scaffold


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

Finally, to better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens.

In [54]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] a [PAD] | | [PAD] [PAD] l l [PAD] [PAD] [PAD] [PAD] a a w n n n n [PAD] d d [PAD] r r r e [PAD] y y [PAD] [PAD] | | [PAD] s h h o u u l d | | b e e e [PAD] | [PAD] e e [PAD] [PAD] [PAD] [PAD] s [PAD] [PAD] t t [PAD] [PAD] [PAD] [PAD] [PAD] a a [PAD] [PAD] [PAD] b l l l i i i [PAD] [PAD] [PAD] s s h h e e e d | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] a n n d d | | a a | | [PAD] [PAD] m [PAD] [PAD] a a [PAD] [PAD] [PAD] [PAD] t t t [PAD] r r r r o o n n n [PAD] | | [PAD] [PAD] a p p [PAD] p p [PAD] [PAD] [PAD] o o o i i n n [PAD] [PAD] t [PAD] [PAD] e e [PAD] d d | [PAD] [PAD] o n n | | t h e e | | [PAD] f f [PAD] e e e e [PAD] [PAD] m m [PAD] [PAD] a a a a l l e e | [PAD] [PAD] [PAD] s s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] i i i [PAD] [PAD] d e e | | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] w

The output should make it a bit clearer how CTC works in practice. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.

I again advise the reader to take a look at [this](https://distill.pub/2017/ctc) very nice blog post to better understand CTC.